In [2]:
require 'sequel'

DB_MIMIC = Sequel.connect(adapter: 'postgres',
                          host: 'postgres',
                          user: 'postgres',
                          password: 'password',
                          database: 'mimic3_demo',
                          search_path: ['mimiciii']
                          )

DB_OLAP = Sequel.connect(adapter: 'postgres',
                          host: 'postgres',
                          user: 'postgres',
                          password: 'password',
                          database: 'mimic3_demo',
                          search_path: ['olap']
                          )

load '../etl/model.rb'
nil

## Schema 0 - Test: Admission

![Cube 0](img/Cube0.png)

#### Time dimension table

In [ ]:
# Creation
DB_OLAP.drop_table?(:admissions_time)
DB_OLAP.create_table(:admissions_time) do
  DateTime :datetime, primary_key: true
  String :day
  String :month
  String :year
end

# Population
Admission.all.uniq.each do |admission|
  time = admission.admittime
  year = time.year
  month = time.month
  day = time.day
  DB_OLAP[:admissions_time].insert(
    datetime: time,
    day: "#{year}-#{month}-#{day}",
    month: "#{year}-#{month}",
    year: year.to_s
  )
end

nil

#### Fact table

In [ ]:
# Creation
DB_OLAP.drop_table?(:admissions)
DB_OLAP.create_table(:admissions) do
  Integer :id, primary_key: true
  DateTime :datetime
  String :type
  String :insurance
  String :location
  Integer :death
end

# Population
Admission.all.uniq.each do |admission|
  DB_OLAP[:admissions].insert(
    id: admission.row_id,
    datetime: admission.admittime,
    type: admission.admission_type,
    insurance: admission.insurance,
    location: admission.admission_location,
    death: admission.hospital_expire_flag
  )
end

nil

## XML implementation